# **SETUP**

## Import

In [2]:
from __future__ import division

import numpy as np
import pickle
import os
from openrec.tf1.legacy import ImplicitModelTrainer
from openrec.tf1.legacy.utils import ImplicitDataset
from openrec.tf1.legacy.utils.evaluators import ImplicitEvalManager
from openrec.tf1.legacy.recommenders import CML, BPR
from openrec.tf1.legacy.utils.evaluators import AUC
from openrec.tf1.legacy.utils.samplers import PairwiseSampler

seed = 76424236
np.random.seed(seed=seed)

folder_name = f"./generated_data/"

if os.path.exists(folder_name) == False:
    os.makedirs(folder_name)

# **FUNCTION DEFINITION**

In [3]:
def calc_metrics(infilename, trainfilename, gamma=0.2, K=10):
    
    infile = open(infilename, 'rb')
    P = pickle.load(infile)
    infile.close()
    NUM_NEGATIVES = P["num_negatives"]
    _NUM_POSs = dict()
    
    for theuser in P["users"]:
        _NUM_POSs[theuser] = len(P["user_items"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(P["user_items"][theuser])[-300:]
        P["results"][theuser] = list(P["results"][theuser])[-300:]
    
    Zui = dict()
    Ni = dict()
    
    # fill in dictionary Ni
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset
    
    # count #users with non-zero item frequencies
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][0 - _NUM_POSs[theuser]:]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break
                
    # fill in dictionary Zui
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][0 - _NUM_POSs[theuser]:]
        pos_scores = P["results"][theuser][0 - _NUM_POSs[theuser]:]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
            
    # calculate per-user scores
    sum_user_auc = 0.0
    sum_user_recall = 0.0
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][0 - _NUM_POSs[theuser]:]:
            if theitem not in Ni:
                continue
            pui = np.power(Ni[theitem], (gamma + 1) / 2.0)
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) / pui
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 / pui
            denominator += 1 / pui
        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator
    
    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }


# **VALIDATE MODEL**

## Load training set

In [4]:
raw_data = dict()
raw_data['train_data'] = np.load(folder_name + "training_arr.npy")
raw_data['val_data'] = np.load(folder_name + "validation_arr.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')
val_dataset = ImplicitDataset(raw_data['val_data'], raw_data['max_user'], raw_data['max_item'], name='Val')

## Define model

In [5]:
# Avoid tensorflow using cached embeddings
import tensorflow as tf
tf.compat.v1.reset_default_graph()

cml_model = CML(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(), 
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=cml_model, sampler=sampler,
                                     eval_save_prefix=folder_name+"yahoo",
                                     item_serving_size=500)
auc_evaluator = AUC()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






2024-04-24 10:08:41.146740: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2024-04-24 10:08:41.149010: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 4192050000 Hz
2024-04-24 10:08:41.149671: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x556ae89b1a60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-04-24 10:08:41.149687: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


## Validate

In [6]:
cml_model.load(folder_name+"cml-yahoo")

model_trainer._eval_manager = ImplicitEvalManager(evaluators=[auc_evaluator])
model_trainer._num_negatives = 300
model_trainer._exclude_positives([train_dataset, val_dataset])
model_trainer._sample_negatives(seed=10)
model_trainer._eval_save_prefix = folder_name+"cml-yahoo-val-new"
model_trainer._evaluate_partial(val_dataset)

INFO:tensorflow:Restoring parameters from ./generated_data/cml-yahoo
[Subsampling negative items]


100%|██████████| 7328/7328 [00:02<00:00, 2916.12it/s]   


{'AUC': [0.7395833333333333,
  0.96,
  0.9466666666666667,
  0.7983333333333333,
  0.76,
  0.7524183006535949,
  0.9772222222222222,
  0.9833333333333334,
  0.9933333333333333,
  0.9283333333333333,
  0.6522222222222221,
  0.9966666666666667,
  0.9233333333333333,
  0.9266666666666667,
  0.9833333333333333,
  0.9833333333333334,
  0.9933333333333333,
  0.9133333333333333,
  0.7622222222222222,
  0.9966666666666667,
  0.9888888888888889,
  0.9833333333333333,
  0.8366666666666667,
  0.9855555555555556,
  0.9366666666666666,
  1.0,
  0.94,
  0.96,
  0.9933333333333333,
  0.8983333333333333,
  0.9566666666666667,
  1.0,
  0.9866666666666667,
  0.6833333333333333,
  0.9066666666666666,
  0.89,
  0.7993939393939395,
  0.8883333333333333,
  0.9666666666666667,
  0.7244444444444444,
  0.9733333333333334,
  0.9874999999999999,
  0.9816666666666667,
  0.9833333333333333,
  0.9750000000000001,
  0.89,
  0.9811111111111112,
  0.91,
  0.98,
  0.9833333333333333,
  0.6833333333333333,
  0.722222222